# Init

In [ ]:
import os, sys, time

import numpy as np
import matplotlib.pyplot as plt

config = "bruz"
if config == "ur1":
    sys.path.insert(0, "../")
    siepelDir = "C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2"
elif config == "bruz":
    sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
    siepelDir = "/home/pleroy/DATA/CAMILL/WR15_v2"

import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.Probe import Probe
from toolsdiadem.ConfigSer import ConfigSer

In [ ]:
#%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [ ]:
cmap='jet'
configSupport = os.path.join(siepelDir, "wr10_plaqueSupport.ini")
conf = ConfigSer(configSupport)
conf.printConfig()

# Données de la probe

In [ ]:
tmp = np.loadtxt(f"{conf.out_path}/data/S11_probe_interpolated.txt")
S11_probe = tmp[:,1] + 1j * tmp[:,2]
freq_probe = tmp[:,0] / 1e9
d = np.linspace( 0, 300 / ( freq_probe[1] - freq_probe[0] ), freq_probe.size ) * 0.5

# Calcul du fond de chambre
ref: ser4siepel_multiband_scan_elev_F2C.sce

In [ ]:
S11_f2c = ser.computeF2C(conf.base_path, conf.dic_dir, conf.nb_elev, conf.nb_freq, conf.nb_ssb, tag="WR10_")

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(d, ser.dB(np.fft.ifft(S11_f2c)), label="fond de chambre" )
ax.plot(d, ser.dB(np.fft.ifft(S11_probe)), label="probe")
ax.plot(d, ser.dB(np.fft.ifft(S11_f2c - S11_probe ) ), label="fond de chambre - probe")
ax.grid()
ax.legend()
ax.set_xlabel("[m]")
ax.set_ylabel("[dB]")
ax.set_title( "time domain" )

# Sauvegarde des données

In [ ]:
file_f2c = f"{conf.out_path}/data/S11_fond_de_chambre.txt"
tmp[:,1] = np.real(S11_f2c)
tmp[:,2] = np.imag(S11_f2c)
np.savetxt(file_f2c, tmp, header="#GHz real imag")
print(file_f2c)